In [1]:
import pyro
import pyro.distributions as dist
import torch
import matplotlib.pyplot as plt
from pyro.infer import MCMC, NUTS, HMC
import seaborn as sns
from torch.distributions import MultivariateNormal
import pickle
import numpy as np
import os


Bad key "text.kerning_factor" on line 4 in
/Users/chenyilu/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils
import torch.utils.data
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import preprocessing

In [2]:
pip install pyro-ppl

You should consider upgrading via the '/Users/chenyilu/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [16]:
data_measurement = pd.read_csv("MeasuredData.txt", sep=" ")
data_measurement = pd.DataFrame(data_measurement)

In [5]:
class NeuralNet(nn.Module):

    def __init__(self, input_dimension, output_dimension, n_hidden_layers, neurons):
        super(NeuralNet, self).__init__()
        # Number of input dimensions n
        self.input_dimension = input_dimension
        # Number of output dimensions m
        self.output_dimension = output_dimension
        # Number of neurons or units per layer 
        self.neurons = neurons
        # Number of hidden layers 
        self.n_hidden_layers = n_hidden_layers
        # Activation function 
        self.activation = nn.Tanh()
        
        if self.n_hidden_layers != 0:
            self.input_layer = nn.Linear(self.input_dimension, self.neurons)
            self.hidden_layers = nn.ModuleList([nn.Linear(self.neurons, self.neurons) for _ in range(n_hidden_layers-1)])
            self.output_layer = nn.Linear(self.neurons, self.output_dimension)

        else:
            print("Simple linear regression")
            self.linear_regression_layer = nn.Linear(self.input_dimension, self.output_dimension)

    def forward(self, x):
        # The forward function performs the set of affine and non-linear transformations defining the network 
        # (see equation above)
        if self.n_hidden_layers != 0:
            x = self.activation(self.input_layer(x))
            for k, l in enumerate(self.hidden_layers):
                x = self.activation(l(x))
            return self.output_layer(x)
        else:
            return self.linear_regression_layer(x)

def NeuralNet_Seq(input_dimension, output_dimension, n_hidden_layers, neurons):
    modules = list()
    modules.append(nn.Linear(input_dimension, neurons))
    modules.append(nn.Tanh())
    for _ in range(n_hidden_layers):
        modules.append(nn.Linear(neurons, neurons))
        modules.append(nn.Tanh())
    modules.append(nn.Linear(neurons, output_dimension))
    model =  nn.Sequential(*modules)
    return model


In [6]:
my_network = NeuralNet(input_dimension = 2, output_dimension = 1, n_hidden_layers=2, neurons =20)

In [7]:
opt_type = "ADAM"

In [8]:
if opt_type == "ADAM":
    optimizer_ = optim.Adam(my_network.parameters(), lr=0.0001, weight_decay=0)
elif opt_type == "LBFGS":
    optimizer_ = optim.LBFGS(my_network.parameters(), lr=0.1, max_iter=1, max_eval=50000, tolerance_change=1.0 * np.finfo(float).eps)
else:
    raise ValueError("Optimizer not recognized")

In [9]:
def fit(model, training_set, x_validation_, y_validation_, num_epochs, optimizer, p, verbose=True):
    history = [[], []]
    
    # Loop over epochs
    for epoch in range(num_epochs):
        if verbose: print("################################ ", epoch, " ################################")

        running_loss = list([0])
        
        # Loop over batches
        for j, (x_train_, u_train_) in enumerate(training_set):
            
            def closure():
                # zero the parameter gradients
                optimizer.zero_grad()
                # forward + backward + optimize
                u_pred_ = model(x_train_)
                # Item 1. below
                loss = torch.mean(abs(u_pred_.reshape(-1, ) - u_train_.reshape(-1, ))**p)
                # Item 2. below
                loss.backward()
                # Compute average training loss over batches for the current epoch
                running_loss[0] += loss.item()/ len(training_set)
                return loss
            
            # Item 3. below
            optimizer.step(closure=closure)
            
        y_validation_pred_ = model(x_validation_)
        validation_loss = torch.mean((y_validation_pred_.reshape(-1, ) - y_validation_.reshape(-1, ))**p).item()
        history[0].append(running_loss[0])
        history[1].append(validation_loss)
        
        if verbose:
            print('Training Loss: ', np.round(running_loss[0],8))
            print('Validation Loss: ', np.round(validation_loss,8))
            
    print('Final Training Loss: ', np.round(history[0][-1],8))
    print('Final Validation Loss: ', np.round(history[1][-1],8))
    return history

In [11]:
data_training = pd.read_csv("TrainingData.txt", sep=" ")
data_training = pd.DataFrame(data_training)

In [12]:
data_training.shape[0]
validation_size = int(30*data_training.shape[0]/100)
training_size = data_training.shape[0] - validation_size

x_train = data_training.iloc[:training_size,:2]
y_train = data_training.iloc[:training_size,2]

x_val = data_training.iloc[training_size:,0:2]
y_val = data_training.iloc[training_size:,2]

x_val = torch.from_numpy(x_val.values).float()
y_val = torch.from_numpy(y_val.values).float()

batch_size = 5
training_set = DataLoader(torch.utils.data.TensorDataset(torch.from_numpy(x_train.values).float(), torch.from_numpy(y_train.values).float()), batch_size= batch_size, shuffle = True)

In [13]:
n_epochs= 500
history = fit(my_network, training_set, x_val, y_val, n_epochs, optimizer_, p=2, verbose = False)

Final Training Loss:  0.03381264
Final Validation Loss:  0.53162813


In [14]:
def G(inputs): 
    pred_mean = my_network(inputs).detach().reshape(-1,1)[0]
    return pred_mean

In [15]:
mu_prior = 12
sigma_prior = 4
sigma_likelihood = 0.075

def model(t, T):
    # Prior is a gaussian distriubtion with mean 0 and standard deviation 0.1
    prior_u = dist.Normal(mu_prior, sigma_prior)
    u = pyro.sample("u", prior_u)
    u = u.expand(t.shape[0], 1)
    inputs = torch.cat([t, u], 1)
    mean = G(inputs)
    # Likelihood is a gaussian distriubtion with mean G(x,w)=wx and standard deviation 0.1
    likelihood = dist.Normal(mean, sigma_likelihood)
    u_sampled = pyro.sample("obs", likelihood, obs=T)

In [17]:
measured_data_input=torch.Tensor(data_measurement.iloc[:,0].values.astype(np.float).reshape(-1,1))
measured_data_output= torch.Tensor(data_measurement.iloc[:,1].values.astype(np.float).reshape(-1,1))

measured_data_input = torch.from_numpy(data_measurement.iloc[:,0].values.reshape(-1,1)).float()
measured_data_output = torch.from_numpy(data_measurement.iloc[:,1].values.reshape(-1,1)).float()
type(measured_data_input)
type(measured_data_output)

torch.Tensor

In [18]:
G(torch.cat([measured_data_input, measured_data_output], 1).float())

tensor([1.1274])

In [19]:
n_samples = 10000
nuts_kernel = NUTS(model)
posterior = MCMC(nuts_kernel, num_samples=n_samples, warmup_steps=1000, initial_params={"u": torch.tensor(mu_prior)})
posterior.run(measured_data_input, measured_data_output)

Warmup:   0%|          | 0/11000 [00:00, ?it/s]

RuntimeError: only Tensors of floating point dtype can require gradients